# Managing ESG-F facet sets with pyessv

NOTE - this notebook is being run against a previously initialized set of facets

## Setup

### Import libs.

In [ ]:
import os

import pyessv

### Local file system facet directory

In [ ]:
# Have mapped an environment variable upon file system
os.getenv("ESGF_PYESSV_HOME")

### Partition creation

In [ ]:
# Create vocab partition.
partition = pyessv.create_partition(u"esgf.facets", os.getenv("ESGF_PYESSV_HOME"))

### Partition cache

In [ ]:
# Cache loading from file system.
partition.load()

In [ ]:
# Cache count - all facets in all facet sets;
assert partition.get_count() == 6052

# Cache count - facets within single facet set;
assert partition.get_count("model") == 185

# Cache count - single term;
assert partition.get_count("model", "ipsl-cm5a-lr") == 1

In [ ]:
# Cache query - all facet sets;
facet_sets = partition.query()
assert isinstance(facet_sets, dict)

# Cache query - single facet set;
facet_set = partition.query("model")
assert isinstance(facet_set, list)

# Cache query - single facet.
facet = partition.query("model", "ipsl-cm5a-lr")
assert isinstance(facet, pyessv.Facet)

In [ ]:
# Cache listing - all facet sets;
partition.show()

In [ ]:
# Cache listing - single facet set;
partition.show("model")

In [ ]:
# Cache listing - single facet;
partition.show("model", "ipsl-cm5a-lr")

## Term management

### Creation

In [ ]:
# Use partition to create new facet.
facet = partition.create("model", "xxx")
assert isinstance(facet, pyessv.Facet)

In [ ]:
# Invalid facet - facet type has invalid characters.
invalid_facet = partition.create(u"&!@", u"xxx")
assert invalid_facet.is_valid == False
assert len(invalid_facet.errors) == 1

In [ ]:
# Invalid facet - facet name has invalid characters.
invalid_facet = partition.create(u"model", u"x&!")
assert invalid_facet.is_valid == False
assert len(invalid_facet.errors) == 1

In [ ]:
# Invalid facet - facet type & name have invalid characters.
invalid_facet = partition.create(u"&!@", u"x&!")
assert invalid_facet.is_valid == False
assert len(invalid_facet.errors) == 2

### Encoding

In [ ]:
# Encode as python dictionary.
as_dict = facet.as_dict()
assert isinstance(as_dict, dict)

# Encode as JSON.
as_json = facet.as_json()
assert isinstance(as_json, str)

### Persistence

In [ ]:
# Newly created facet is not yet saved to either in memory cache or local disk.
facet = partition.create("model", "xxx")
assert facet.is_cached == False
assert facet.is_written == False

In [ ]:
# Upon saving a facet it is written to both memory cache & file system.
facet.save()
assert facet.is_cached == True
assert facet.is_written == True

## Term Governance

In [ ]:
# What is initial governance status ?
facet.status

In [ ]:
# Accept - auto updates cache & file system.
facet.accept()

In [ ]:
# Reject - auto updates cache & file system.
facet.reject()

In [ ]:
# Deprecate - auto updates cache & file system.
facet.deprecate()

In [ ]:
# Destroy - deleted from cache & file system.
facet_count = partition.get_count(u"model")
facet.destroy()
assert partition.get_count(u"model") == facet_count - 1
assert partition.query(u"model", "xxx") is None

## Facet Parsing

In [ ]:
# Use partition to create new test facet.
facet = partition.create(u"model", u"yyy")

# Set term alternative name.
facet.alternative_name = u"zzz"

# Set term alias.
facet.aliases.append("YYY-1")

# Save term.
partition.save(facet)

In [ ]:
# Parse a name that matches. 
name = "yyy"
parsed_name = partition.parse(u"model", name)
assert name == parsed_name

In [ ]:
# Parse a name that almost matches. 
name = "yYY"
parsed_name = partition.parse(u"model", name)
assert name != parsed_name
assert parsed_name == "yyy"

In [ ]:
# Parse an alternative name.
alt_name = "zzz"
parsed_name = partition.parse(u"model", alt_name)
assert alt_name != parsed_name
assert parsed_name == "yyy"

In [ ]:
# Parse an alias.
alias = "YYY-1"
parsed_name = partition.parse(u"model", alias)
assert alias != parsed_name
assert parsed_name == "yyy"

In [ ]:
# Parsing an invalid alias returns None.
alias = "YYY-1SSS"
parsed_name = partition.parse(u"model", alias)
assert parsed_name is None

In [ ]:
# Destroy test facet.
facet.destroy()